#### 제주도 핫 플레이스 인스타그램에서 크롤링하기

#### matplotlib 한글설정

#### 셀레니움 설정



In [2]:
import matplotlib
#from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
import seaborn as sns

font_path = 'C:/Windows/Fonts/HANBatang.ttf'
font = font_manager.FontProperties(fname=font_path).get_name() #나눔고딕, NanumGothic
rc('font', family=font)
rcParams['axes.unicode_minus'] = False

In [9]:
# 셀레니움
#모둘 임포트
from selenium import webdriver

driver = webdriver.Chrome()

url = 'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91/'
driver.get(url)

#### 첫번째 게시글 찾아서 설정하기

In [10]:
from selenium.webdriver.common.by import By
import time

def select_first_post(driver):
    first = driver.find_element(By.CLASS_NAME, '_aagw')
    first.click()
    time.sleep(3)

select_first_post(driver)

##### 게시글 정보 가져오기

In [23]:
import re # RegularExpression 정규식 처리 모듈
from bs4 import BeautifulSoup
import unicodedata  # 유니코드 데이터 처리 모듈

def get_content(driver):
    # 1. 데이터가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 본문내용 찾아오기
    try:
        content = soup.select('div._a9zr')[0].text   # _a9zr은 추후 변경됨(인스타그램 정책)
    except:
        content = ''

    # 3. 본문내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 4. 작성일자 정보
    try:
        date = soup.select('time._aaqe')[0]['datetime'][:10] # _aaqe은 추후 변경됨(인스타그램 정책)
    except:
        date = ''

    # 5. 좋아요수 가져오기
    try:
        # x193iq5w등은 추후 변경됨(인스타그램 정책)
        like = int(soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp > span')[0].text)
    except:
        like = 0

    # 6. 위치정보
    # 정규식 학습필수
    # _aagv 마지막 인덱스를 찾으면 된다
    # imgs = soup.select('div._aatk._aatl._aatm div._aagv > img')[0]['alt']
    address = ''
    try:
        regex = r'(\w+[시,읍]\s*)?' \
                r'?(\w+\d*\w*[로,리,길]\s*)' \
                r'?(\w*\d+-?\d*)?'

        place = re.findall(regex, content)
        for pl in place:
            if pl[0] != '' and pl[1] != '' and pl[2] != '':
                address = f'{pl[0]}{pl[1]}{pl[2]}'
    except:
        address = ''

    # 7. 정보 저장
    data = [content, date, like, address, tags]
    return data

get_content(driver)

['la_._es_i_ato_(리그램) @dotoree_jeju두둥 ! 맛집러들 다 모여📢더많은 정보를 알고싶다면 !? 🌟제주 핫플 찐맛집 !! 감성적인 인테리어와 소바전문점 도토리키친 본점 ✔ 깔끔한 내부 취향저격 비주얼에 놀라고, 맛은 기절 할 정도로 맛있는 찐맛탱 제주 맛집 !! 👏👏✔ 청정제주에서 1년에 한달간만 수확할 수 있는 귀한 청귤소바😍몽글몽글한 계란과 톳유부초밥의 만남👍👍제발 안 먹어본 사람 없게 해주세요~~😍제주 필수 코스 〚도토리키친〛 @ @같이갈친구태그□ 도토리키친 본점□ 제주 제주시 북성로 59 1층□ 주차 무료입니다!! □ 영업시간 10:30~16:30 (16:00라스트오더)□ 0507-1463-1021□주문 마감은 4시입니다~재료소진시 조기마감합니다. ..#제주공항근처맛집 #제주공항근처맛집추천 #제주맛집#제주맛집추천 #제주맛집투어 #제주맛집탐방 #제주도맛집 #제주도맛집추천 #제주도맛집베스트 #제주도맛집투어 #제주시맛집 #제주시맛집추천 #제주 #제주여행 #제주도 #청귤소바6시간',
 '2023-06-29',
 0,
 '제주시 북성로 59',
 ['#제주공항근처맛집',
  '#제주공항근처맛집추천',
  '#제주맛집',
  '#제주맛집추천',
  '#제주맛집투어',
  '#제주맛집탐방',
  '#제주도맛집',
  '#제주도맛집추천',
  '#제주도맛집베스트',
  '#제주도맛집투어',
  '#제주시맛집',
  '#제주시맛집추천',
  '#제주',
  '#제주여행',
  '#제주도',
  '#청귤소바6시간']]

#### 다음게시글 열기

In [25]:
def move_next(driver):
    right =driver.find_element(By.CSS_SELECTOR,('._aaqg._aaqh'))
    right.find_element(By.XPATH, '//button')
    right.click()
    time.sleep(3)

move_next(driver)

#### 자동 크롤링 구현

In [43]:
# 모듈 임포트 생략

driver = webdriver.Chrome()
driver.get('https://www.instagram.com')
time.sleep(2.0)

In [45]:
# 아이디, 패스워드 입력해서 자동 로그인하기
id = 'gusqls7748@naver.com'
input_id = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[0]
input_id.clear()
input_id.send_keys(id)
password = 'S@ubin3359'
input_pw = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(2.0)



In [48]:
#알림설정 버튼 클릭
alarm_btn = driver.find_element(By.CSS_SELECTOR, '._a9--._a9_0')
alarm_btn.click()

In [49]:
# 제주도 맛집 찾기 함수
def insta_searching(word):
    url = f'https://www.instagram.com/explore/tags/{word}'
    return(url)

In [50]:
#크롤링 시작하기
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)
time.sleep(10.0)

select_first_post(driver)

results = []

target = 50
for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2.0)
        move_next(driver)

print(len(results))

50
